In [10]:
from torch_loader import *
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
from pathlib import Path
import torchvision
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn as nn

In [7]:
batch_size = 32
train_size = int(0.8 * len(remaining_dataset))
test_size = len(remaining_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(remaining_dataset,
                                                            [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)#, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)#, num_workers=4)

In [8]:
images, labels = next(iter(train_loader))
print(images.shape)

torch.Size([128, 3, 224, 224])


In [ ]:
def train(model, optimizer, loss_fn, num_epochs):
    # def loss_fun(output, target):
    #     return F.cross_entropy(output, target)
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': [],
              'y_pred': [],
              'y_true': [],
              'SEM': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        #For each epoch
        train_correct = 0
        train_loss = []
        for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            data, target = data.to(device), target.to(device)
            
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(data)
                       
            #Compute the loss
            loss = loss_fn(target, output)
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()


            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(dim=1)
            train_correct += (target==predicted).sum().cpu().item()
        #Comput the test accuracy
        test_loss = []
        test_correct = 0
        model.eval()
        y_pred = []
        y_true = []
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            test_loss.append(loss_fn(target, output).cpu().item())
            predicted = output.argmax(1)
            test_correct += (target==predicted).sum().cpu().item()
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(target.cpu().numpy())
        out_dict['train_acc'].append(train_correct/len(train_dataset))
        out_dict['test_acc'].append(test_correct/len(test_dataset))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['test_loss'].append(np.mean(test_loss))
        out_dict['y_pred'].append(y_pred)
        out_dict['y_true'].append(y_true)
        std = 1 - accuracy_score(y_true, y_pred)
        sem = std / np.sqrt(len(y_true))
        sem_p = (sem / 1)
        out_dict['SEM'].append(sem_p)
        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%\t", f"SEM: {out_dict['SEM'][-1]*100:.1f}%\t")
        torch.save(model, 'model_CNN.pth')
    return out_dict


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def CrossEntropy(target, output):
    return F.cross_entropy(output, target)

In [ ]:
import torchvision.models as models
model = models.vgg16(pretrained=True).to(device)
for param in model.parameters():
    param.requires_grad = False

num_features = model.classifier[-1].in_features
model.classifier[6] = nn.Linear(num_features, 5)
model = model.to(device)
optimizer = torch.optim.AdamW(lr=1e-3, params=model.parameters(), weight_decay=0.1)
output_dict = train(model, optimizer, CrossEntropy, num_epochs=50)

In [ ]:
train_acc = output_dict['train_acc']
test_acc = output_dict['test_acc']
train_loss = output_dict['train_loss']
test_loss = output_dict['test_loss']
y_pred = output_dict['y_pred']
y_true = output_dict['y_true']
epochs = range(1, len(train_acc) + 1)

plt.subplot(2, 1, 1)
plt.plot(epochs, train_acc, label='Train Accuracy')
plt.plot(epochs, test_acc, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy plot')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, train_loss, label='Train Loss')
plt.plot(epochs, test_loss, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss plot')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model.eval()
y_pred_validation = []
y_true_validation = []

x_test, y_test = x_test.to(device), y_test.to(device)
with torch.no_grad():
    output = model(x_test)
    predicted = output.argmax(dim = 1)
y_pred_validation.extend(predicted.cpu().numpy())
y_true_validation.extend(y_test.cpu().numpy())


std_val = 1 - accuracy_score(y_true_validation, y_pred_validation)
sem_val = std_val / np.sqrt(len(y_true_validation))
sem_p_val = (sem_val / 1) * 100
print(accuracy_score(y_true_validation, y_pred_validation) * 100, sem_p_val)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_true_validation, y_pred_validation, labels=[0,1,2,3,4], normalize = 'true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = ["stage W", "Stage 1", "Stage 2", "Stage 3/4", "Stage R"])
disp.plot()
plt.show()


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true_validation, y_pred_validation, target_names=['sleep stage W', 'sleep Stage 1', 'sleep Stage 2', 'Sleep Stage 3/4', 'Sleep Stage R']))